In [ ]:
import pandas as pd
import numpy as np
import glob

Load and process the csvs

In [ ]:
csv_paths = glob.glob('./*.csv')
df_list = []

for path in csv_paths:
    df = pd.read_csv(path)
    required_cols = {'mode', 'distance_meters', 'duration_seconds', 'duration_in_traffic'}
    if required_cols.issubset(df.columns):
        df_list.append(df)

# === Combine All Valid CSVs ===
combined_df = pd.concat(df_list, ignore_index=True)

Emission Factors

In [ ]:
emission_factors = {
    'driving': 0.192,# CHECk
    'transit': 0.105,
    'bicycling': 0.0,
    'walking': 0.0
}


Compute Final Duration

In [ ]:
def compute_final_duration(row):
    if row['mode'] == 'driving' and not pd.isna(row['duration_in_traffic']):
        return row['duration_in_traffic']
    return row['duration_seconds']

combined_df['duration_final'] = combined_df.apply(compute_final_duration, axis=1)

Compute Emissions

In [ ]:
combined_df['emissions'] = combined_df.apply(
    lambda row: emission_factors.get(row['mode'], 0) * row['distance_meters'],
    axis=1
)

Scoring Function

In [ ]:
T_max = 1800
E_max = 1000
t_weight = 15
e_weight = 1
alpha, beta = 0.015, 0.015



def route_score(t, e, T=T_max, E=E_max, a=t_weight, b=e_weight, alpha=0.04, beta=0.015):
    t_ratio = t / T
    e_ratio = e / E
    penalty_t = alpha * max(0, t - T)
    penalty_e = beta * max(0, e - E)
    return a * t_ratio + b * e_ratio + penalty_t + penalty_e


Apply Score

In [ ]:
combined_df['score'] = combined_df.apply(
    lambda row: route_score(row['duration_final'], row['emissions']),
    axis=1
)


Save Final Output

In [ ]:
combined_df.to_csv("enriched_routes_with_score4.csv", index=False)

print(combined_df)


            day   time   timestamp        origin_place  origin_lat  \
0      8/4/2025   6:00  1754312400       Westlake Park   47.610949   
1      8/4/2025   6:00  1754312400       Westlake Park   47.610949   
2      8/4/2025   6:00  1754312400       Westlake Park   47.610949   
3      8/4/2025   6:00  1754312400       Westlake Park   47.610949   
4      8/4/2025   7:00  1754316000       Westlake Park   47.610949   
...         ...    ...         ...                 ...         ...   
6659  8/10/2025  21:00  1754884800  Seattle Art Museum   47.607515   
6660  8/10/2025  22:00  1754888400  Seattle Art Museum   47.607515   
6661  8/10/2025  22:00  1754888400  Seattle Art Museum   47.607515   
6662  8/10/2025  22:00  1754888400  Seattle Art Museum   47.607515   
6663  8/10/2025  22:00  1754888400  Seattle Art Museum   47.607515   

      origin_lng                                  destination_place  \
0    -122.337021  International District/Chinatown Station - lig...   
1    -122.337021 